In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [29]:
# load mixed (measured) spectra
mixed_df = pd.read_csv("C:\\Users\\zhalem\\Downloads\\SpectralUnmixing.jl-v0.2.7\\emit-sds-SpectralUnmixing.jl-3e47809\\examples\\mixed_onevariety_1108.csv")

# save metadata
sample_ids = mixed_df.iloc[:, 0] # First column = sample name
mixed_df = mixed_df.iloc[:, 1:] # Remaining columns = wavelengths
mixed_df.columns = mixed_df.columns.astype(float) # Force wavelength columns to numeric
R_obs = mixed_df.to_numpy()
wavelengths = mixed_df.columns.to_numpy()


In [30]:
# load in your endmember library
endmember_df = pd.read_csv("C:\\Users\\zhalem\\Downloads\\SpectralUnmixing.jl-v0.2.7\\emit-sds-SpectralUnmixing.jl-3e47809\\examples\\endmember_onevariety_1108.csv")

# Save metadata
endmember_type = endmember_df.iloc[:, 0].values   # soil / leaf
endmember_ids  = endmember_df.iloc[:, 1].values   # spectra ID
endmember_df = endmember_df.iloc[:, 2:] # Keep only wavelength columns
endmember_df.columns = endmember_df.columns.astype(float) # Force wavelength columns to numeric
E = endmember_df.to_numpy()


In [31]:
# load in your unmixed spectra
unmixed_df = pd.read_csv("C:\\Users\\zhalem\\Downloads\\SpectralUnmixing.jl-v0.2.7\\emit-sds-SpectralUnmixing.jl-3e47809\\examples\\final_unmixed_1108.csv")

# Save metadata
unmixed_ids = unmixed_df.iloc[:, 0]
unmixed_df = unmixed_df.iloc[:, 1:]
unmixed_df.columns = unmixed_df.columns.astype(float)
R_recon = unmixed_df.to_numpy()


In [32]:
# load in your unmixing fractions (leaf and soil)
fractions_df = pd.read_csv("C:\\Users\\zhalem\\Downloads\\SpectralUnmixing.jl-v0.2.7\\emit-sds-SpectralUnmixing.jl-3e47809\\examples\\soil_leaf_fractions_1108.csv")

#fraction_ids = fractions_df.iloc[:, 0]
#fractions_df = fractions_df.iloc[:, 1:]
F_hat = fractions_df.to_numpy()
fraction_names = fractions_df.columns.tolist()

## R^2 function

R^2 = 1 − (∑(a − b)^2) / (∑(b − a)^2)

This formula treats a (the unmixed spectrum) as a prediction and b (the reference endmember) as the "truth". It computes the sum of squares differences point by point, normalized by the variance of b. 

It is sensitive to absolute scaling, therefore if a is twice as high as b in reflectance (magnitude), R^2 will drop drastically, even if the shape is perfectly identical. This method captures how well reflectance values match, not overall spectral similarity in shape. 

This method does not account for scaling baseline differences, which we expect here given each spectra has different soil fractions, and can even produce negative R^2 values when the slope differs.  

In [33]:
# Functions

def r2_spectrum(a, b):
    a = np.asarray(a, dtype=np.float64)
    b = np.asarray(b, dtype=np.float64)
    ss_tot = np.sum((b - np.mean(b))**2)
    if ss_tot == 0:
        return 0.0
    ss_res = np.sum((a - b)**2)
    return 1 - ss_res / ss_tot

def sam(a, b):
    a = np.asarray(a, dtype=np.float64)
    b = np.asarray(b, dtype=np.float64)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    if norm_a == 0 or norm_b == 0:
        return np.pi / 2
    cos_angle = np.clip(np.dot(a, b) / (norm_a * norm_b), -1.0, 1.0)
    return np.arccos(cos_angle)


In [34]:
# Keep only leaf endmembers

leaf_mask = endmember_type == "leaf"
E_leaf = E[leaf_mask]


# Align wavelengths across mixed, endmembers, reconstructed
common_waves = np.intersect1d(wavelengths, endmember_df.columns.to_numpy())
idx_recon = [np.where(wavelengths == w)[0][0] for w in common_waves]
idx_endm  = [np.where(endmember_df.columns == w)[0][0] for w in common_waves]

R_recon = R_recon[:, idx_recon]
E_leaf  = E_leaf[:, idx_endm]
wavelengths = common_waves

# Remove NaN bands in reconstructed spectra
valid_mask = ~np.isnan(R_recon).any(axis=0)
R_recon = R_recon[:, valid_mask]
E_leaf  = E_leaf[:, valid_mask]
wavelengths = wavelengths[valid_mask]

print(f"Aligned number of bands: {R_recon.shape[1]}")


Aligned number of bands: 1909


In [35]:
# Compute mean R² and min SAM per unmixed leaf spectrum

mean_r2_list = []
min_sam_list = []

for leaf_spectrum in R_recon:
    r2_vals = [r2_spectrum(leaf_spectrum, leaf_end) for leaf_end in E_leaf]
    mean_r2_list.append(np.mean(r2_vals))
    
    sam_vals = [sam(leaf_spectrum, leaf_end) for leaf_end in E_leaf]
    min_sam_list.append(np.min(sam_vals))

mean_r2_list = np.array(mean_r2_list)
min_sam_list = np.array(min_sam_list)

In [36]:
# Save results

results_df = pd.DataFrame({
    'Sample_ID': unmixed_ids,
    'Mean_R2': mean_r2_list,
    'Min_SAM': min_sam_list,
    'Soil_Fraction': F_hat[:, fraction_names.index("soil")]
})

print(results_df.head())
print(f"Mean R² across samples: {mean_r2_list.mean():.3f} ± {mean_r2_list.std():.3f}")
print(f"Mean SAM across samples: {min_sam_list.mean():.3f} ± {min_sam_list.std():.3f}")

                   Sample_ID   Mean_R2   Min_SAM Soil_Fraction
0  14deg_plot_11_08_0000 (2)  0.147374  0.094245           0.0
1      14deg_plot_11_08_0000 -0.082021  0.057214      0.226185
2  14deg_plot_11_08_0001 (2)  0.157973  0.088967           0.0
3      14deg_plot_11_08_0001  0.471963  0.065431      0.157576
4  14deg_plot_11_08_0002 (2)  0.180459  0.093741           0.0
Mean R² across samples: 0.203 ± 0.165
Mean SAM across samples: 0.080 ± 0.016


## R^2 Function

b ≈ β0 + β1 * a

This method of computing R^2 fits a linear regression of b on a, and then computes the R^2 of the fit. β1 (scaling) and β0 (baseline shift) between a and b. If the shapes of the spectra are identical, but a is scaled or offset, R^2 will still be high. This calculation focuses on spectral shape similarity, not the reflectance values. 

If the goal is to understand differences in absolute reflectance, this can overestimate "fit quality" as it ignores absolute magnitude differences. 

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Functions
def r2_regression(a, b):
    a = np.asarray(a, dtype=np.float64)
    b = np.asarray(b, dtype=np.float64)

    # Handle constant reference
    if np.all(b == b[0]):
        return 0.0

    # Fit linear regression: b = beta0 + beta1 * a
    A = np.vstack([a, np.ones_like(a)]).T
    beta1, beta0 = np.linalg.lstsq(A, b, rcond=None)[0]

    # Predicted values
    b_hat = beta0 + beta1 * a

    # Compute R²
    ss_tot = np.sum((b - np.mean(b))**2)
    ss_res = np.sum((b - b_hat)**2)
    return 1 - ss_res / ss_tot

def sam(a, b):
    a = np.asarray(a, dtype=np.float64)
    b = np.asarray(b, dtype=np.float64)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    if norm_a == 0 or norm_b == 0:
        return np.pi / 2
    cos_angle = np.clip(np.dot(a, b) / (norm_a * norm_b), -1.0, 1.0)
    return np.arccos(cos_angle)


# Keep only leaf endmembers
leaf_mask = endmember_type == "leaf"
E_leaf = E[leaf_mask]


# Align wavelengths across mixed, endmembers, reconstructed

common_waves = np.intersect1d(wavelengths, endmember_df.columns.to_numpy())
idx_recon = [np.where(wavelengths == w)[0][0] for w in common_waves]
idx_endm  = [np.where(endmember_df.columns == w)[0][0] for w in common_waves]

R_recon = R_recon[:, idx_recon]
E_leaf  = E_leaf[:, idx_endm]
wavelengths = common_waves

# Remove NaN bands in reconstructed spectra
valid_mask = ~np.isnan(R_recon).any(axis=0)
R_recon = R_recon[:, valid_mask]
E_leaf  = E_leaf[:, valid_mask]
wavelengths = wavelengths[valid_mask]

print(f"Aligned number of bands: {R_recon.shape[1]}")


# Compute regression-based R² and min SAM per unmixed leaf spectrum
mean_r2_list = []
min_sam_list = []

for leaf_spectrum in R_recon:
    r2_vals = [r2_regression(leaf_spectrum, leaf_end) for leaf_end in E_leaf]
    mean_r2_list.append(np.mean(r2_vals))
    
    sam_vals = [sam(leaf_spectrum, leaf_end) for leaf_end in E_leaf]
    min_sam_list.append(np.min(sam_vals))

mean_r2_list = np.array(mean_r2_list)


# Save results
results_df = pd.DataFrame({
    'Sample_ID': unmixed_ids,
    'Mean_R2': mean_r2_list,
    'Min_SAM': min_sam_list,
    'Soil_Fraction': F_hat[:, fraction_names.index("soil")]
})

print(results_df.head())
print(f"Mean R² across samples: {mean_r2_list.mean():.3f} ± {mean_r2_list.std():.3f}")


Aligned number of bands: 1909
                   Sample_ID   Mean_R2   Min_SAM Soil_Fraction
0  14deg_plot_11_08_0000 (2)  0.952222  0.094245           0.0
1      14deg_plot_11_08_0000  0.973932  0.057214      0.226185
2  14deg_plot_11_08_0001 (2)  0.953688  0.088967           0.0
3      14deg_plot_11_08_0001  0.972095  0.065431      0.157576
4  14deg_plot_11_08_0002 (2)  0.954042  0.093741           0.0
Mean R² across samples: 0.961 ± 0.012
